In [1]:
#!pip install --upgrade git+https://github.com/EmGarr/kerod.git

In [2]:
#%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    print('GPU device not found')
else:
    print('Found GPU at: {}'.format(device_name))

GPU device not found


In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import  functools
import  tensorflow as tf
import  tensorflow_datasets as tfds
from    tensorflow.keras.utils import to_categorical
import  matplotlib.pyplot as plt
from    tensorflow.keras import layers
from    tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
import pandas as pd
from    tqdm import tqdm
import  time
from    sklearn.metrics import accuracy_score


In [5]:
from Helper import Config, ImagesInfo, Logger, Client, TimeKeeper
from Helper import read_image

In [6]:
data_dir='/home/suphale/coco'
N_LABELS = 80
N_EPOCHS = 1
TRAIN_MODE = False
split_train = "train[:1%]"
split_val = "validation[:1%]"
h_image_height = 299
h_image_width = 299

SPLIT_LAYER = 4


In [7]:
tk = TimeKeeper()
cfg = Config()
client = Client(cfg)
imagesInfo = ImagesInfo(cfg)

In WSL


# Load and train the network


In [8]:
from tensorflow import keras  # or import keras for standalone version
from tensorflow.keras.layers import Input

# org_model = tf.keras.models.load_model(cfg.temp_path + '/model')
org_model = tf.keras.models.load_model(cfg.temp_path + '/model_2021_07_21')



In [9]:
model_config = org_model.get_config()

In [10]:
max_layer_index = len(model_config['layers']) - 1

In [11]:

new_head_model_config = {}
new_head_model_config['name'] = 'head_model'
new_head_model_config['layers'] = []
new_head_model_config['input_layers'] = [[model_config['layers'][0]['name'],0,0]]
new_head_model_config['output_layers'] = [[model_config['layers'][SPLIT_LAYER-1]['name'],0,0]]

for index in range(SPLIT_LAYER):
    print("%d %s" % (index, model_config['layers'][index]['name']) )
    new_head_model_config['layers'].append(model_config['layers'][index])

0 input_1
1 conv2d
2 batch_normalization
3 activation


In [12]:
import copy

new_tail_model_config = copy.deepcopy(model_config)
new_tail_model_config['name'] = 'tail_model'
new_tail_model_config['input_layers'] = [[model_config['layers'][SPLIT_LAYER]['name'],0,0]]
new_tail_model_config['output_layers'] = [[model_config['layers'][max_layer_index]['name'],0,0]]

In [13]:
# tuple(org_model.layers[0].output.shape)

In [14]:
new_input_layer = {
                      'name': 'new_input',
                      'class_name': 'InputLayer',
                      'config': {
                          'batch_input_shape': tuple(org_model.layers[SPLIT_LAYER].output.shape),
                          'dtype': 'float32',
                          'sparse': False,
                          'name': 'new_input'
                      },
                      'inbound_nodes': []
                  }
new_tail_model_config['layers'][0] = new_input_layer

In [15]:
for index in range(1,SPLIT_LAYER):
    print("%d %s" % (index, new_tail_model_config['layers'][1]['name']) )
    # new_tail_model_config['layers'].append(model_config['layers'][index])
    new_tail_model_config['layers'].pop(1)

1 conv2d
2 batch_normalization
3 activation


In [16]:
# input_layer_name = model_config['layers'][0]['name']
new_tail_model_config['layers'][1]['inbound_nodes'] = [[['new_input', 0, 0, {}]]]
new_tail_model_config['input_layers'] = [['new_input', 0, 0]]

In [17]:
import pprint
with open(cfg.temp_path + '/model_config.txt','w') as fh:
    # Pass the file handle in as a lambda function to make it callable
    # fh.write(str(model_config))
    pp = pprint.PrettyPrinter(indent=4, stream=fh)
    pp.pprint(str(model_config))
with open(cfg.temp_path + '/new_head_model_config.txt','w') as fh:
    # Pass the file handle in as a lambda function to make it callable
    # fh.write(str(new_head_model_config))
    pp = pprint.PrettyPrinter(indent=4, stream=fh)
    pp.pprint(str(new_head_model_config))
with open(cfg.temp_path + '/new_tail_model_config.txt','w') as fh:
    # Pass the file handle in as a lambda function to make it callable
    # fh.write(str(new_head_model_config))
    pp = pprint.PrettyPrinter(indent=4, stream=fh)
    pp.pprint(str(new_tail_model_config))


In [18]:
new_head_model = org_model.__class__.from_config(new_head_model_config, custom_objects={})

In [19]:
new_tail_model = org_model.__class__.from_config(new_tail_model_config, custom_objects={})

In [20]:
with open(cfg.temp_path + '/org_model.txt','w') as fh:
    # Pass the file handle in as a lambda function to make it callable
    org_model.summary(print_fn=lambda x: fh.write(x + '\n'), line_length=150)

with open(cfg.temp_path + '/new_head_model.txt','w') as fh:
    # Pass the file handle in as a lambda function to make it callable
    new_head_model.summary(print_fn=lambda x: fh.write(x + '\n'), line_length=150)

with open(cfg.temp_path + '/new_tail_model.txt','w') as fh:
    # Pass the file handle in as a lambda function to make it callable
    new_tail_model.summary(print_fn=lambda x: fh.write(x + '\n'), line_length=150)


In [21]:
for index, layer in enumerate(org_model.layers[:SPLIT_LAYER]):
    # print("[%d] %s %s" % (index, layer.name, str(np.shape(weight))))
    weight = layer.get_weights()
    new_head_model_layer = new_head_model.layers[index]
    new_head_model_layer.set_weights(weight)

In [22]:
for index, layer in enumerate(org_model.layers[SPLIT_LAYER:max_layer_index+1]):
    weight = layer.get_weights()
    # print("[%d] %s %s" % (index, layer.name, str(np.shape(weight))))
    new_tail_model_layer = new_tail_model.layers[index+1]
    # print("[%d] %s %s" % (index, new_tail_model_layer.name, str(np.shape(weight))))
    new_tail_model_layer.set_weights(weight)

In [23]:
new_head_model.save(cfg.temp_path + '/new_head_model')
new_tail_model.save(cfg.temp_path + '/new_tail_model')

INFO:tensorflow:Assets written to: ./temp/new_head_model/assets
INFO:tensorflow:Assets written to: ./temp/new_tail_model/assets


In [24]:
def process_predictions(ground_truth, prediction_tensor):
    n = tf.squeeze(prediction_tensor).numpy()
    df = pd.DataFrame(columns=['id_index','probability'])
    predictions_str = ''
    top_predictions = []
    index = 0
    for x in n:
        if x > cfg.PREDICTIONS_THRESHOLD:
            top_predictions.append(index)
            predictions_str += "%s(%.2f) " % (imagesInfo.classes[index],x)
            df = df.append({'id_index':int(index), 'probability':x},ignore_index = True)
        index += 1

    df = df.sort_values('probability', ascending=False)
    sorted_predictions = df['id_index'].tolist()
    sorted_predictions = [int(x) for x in sorted_predictions]

    ground_truth_length = len(ground_truth)
    predictions_length = len(sorted_predictions)

    aligned_predictions = [-1] * ground_truth_length
    TP = 0
    for i in range(ground_truth_length):
        if(ground_truth[i] in sorted_predictions):
            aligned_predictions[i] = ground_truth[i]
            TP += 1
    accuracy = accuracy_score(ground_truth, aligned_predictions)

    top_1_accuracy = 0.0
    top_5_accuracy = 0.0
    precision = 0
    recall = 0
    if(predictions_length > 0):
        if(sorted_predictions[0] in ground_truth):
            top_1_accuracy = 1.0
        for i in range(5):
            if((i < predictions_length) and (sorted_predictions[i] in ground_truth)):
                top_5_accuracy = 1.0

        precision = TP / predictions_length
    recall = TP / ground_truth_length
    return accuracy, top_1_accuracy,top_5_accuracy,precision,recall, top_predictions, predictions_str


In [25]:
def evaluate_classification(image):
    temp_input = tf.expand_dims(read_image(image), 0) 
    h = new_head_model(temp_input)
    s = new_tail_model(h)
    return s

In [26]:
total_test_images= 100
total_time = 0.0
max_test_images = total_test_images
df = pd.DataFrame(columns=['img_path','ground_truth', 'top_predict', 'Prediction', 'accuracy', 'top_1_accuracy', 'top_5_accuracy', 'precision', 'recall', 'time'])

for i in tqdm(range(max_test_images)):
    Logger.debug_print("")
    img_path = imagesInfo.all_img_vector[i]
    # plt.imshow(image)
    ground_truth = imagesInfo.get_segmentation_id_indexes(img_path)

    test_image = img_path

    t0= time.perf_counter()
    s = evaluate_classification(test_image)
    t1 = time.perf_counter() - t0
    total_time = total_time + t1

    accuracy, top_1_accuracy,top_5_accuracy,precision,recall, top_predictions, predictions_str = process_predictions(ground_truth,s)
    df = df.append(
        {'image':img_path, 
        'ground_truth':(str(imagesInfo.get_segmentation_texts(ground_truth))),
        'top_predict':str(top_predictions),
        'Prediction':predictions_str,
        'accuracy':accuracy,
        'top_1_accuracy':top_1_accuracy,
        'top_5_accuracy':top_5_accuracy,
        'precision':precision,
        'recall':recall,
        'time':t1,
        },
        ignore_index = True)
    truth_str = ' '.join([str(elem) for elem in imagesInfo.get_segmentation_texts(ground_truth)])
    Logger.debug_print("ground_truth  : %s" % (truth_str))
    Logger.debug_print("Prediction    : %s" % (predictions_str))

df.to_csv(cfg.temp_path + '/results_'+cfg.timestr+'.csv')
av_column = df.mean(axis=0)

Logger.milestone_print("accuracy        : %.2f" % (av_column.accuracy))
Logger.milestone_print("top_1_accuracy  : %.2f" % (av_column.top_1_accuracy))
Logger.milestone_print("top_5_accuracy  : %.2f" % (av_column.top_5_accuracy))
Logger.milestone_print("precision       : %.2f" % (av_column.precision))
Logger.milestone_print("recall          : %.2f" % (av_column.recall))
Logger.milestone_print("time            : %.2f" % (av_column.time))

100%|██████████| 100/100 [00:54<00:00,  1.82it/s]

accuracy        : 0.53
top_1_accuracy  : 0.70
top_5_accuracy  : 0.90
precision       : 0.68
recall          : 0.53
time            : 0.52
